In [0]:
df = spark.read.format("parquet").load("abfss://bronze@ssdatalakeeteproject.dfs.core.windows.net/orders")
display(df)

In [0]:
print("Row count:", df.count())

In [0]:
df = df.withColumnRenamed("_rescued_data","rescued_data")

In [0]:
df = df.drop("rescued_data")

In [0]:
from pyspark.sql.functions import to_timestamp
df = df.withColumn("order_date", to_timestamp("order_date"))


In [0]:
from pyspark.sql.functions import year
from pyspark.sql.functions import col
df = df.withColumn("year", year(col("order_date")))
                                    

In [0]:
from pyspark.sql.functions import dense_rank, desc
from pyspark.sql.window import Window
df1 = df.withColumn("flag",dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

df1.display()

In [0]:
from pyspark.sql.functions import rank, desc
df1 = df1.withColumn( "rank_flag",rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df1.display()


In [0]:
from pyspark.sql.functions import row_number
df1 = df1.withColumn("row_flag",row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df1.display()

In [0]:

 class windows:

    def dense_rank(self,df):
        df_dense_rank = df.withColumn("flag",dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_dense_rank

    def rank(self,df):
        df_rank = df.withColumn("rank_flag",rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_rank

    def row_number(self,df):
        df_row_number = df.withColumn("row_flag",row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_row_number

In [0]:

df_new = df
df_new.display() 


In [0]:
obj = windows()


In [0]:
df_result = obj.dense_rank(df_new)
df_result.display()
df_result = obj.rank(df_new)
df_result.display()
df_result = obj.row_number(df_new)
df_result.display() 

In [0]:
df.write.format("delta").mode("overwrite")\
    .save("abfss://silver@ssdatalakeeteproject.dfs.core.windows.net/orders")
df.display()



In [0]:
%sql
create table if not exists databricks_catalog.silver.orders_silver
using delta
location "abfss://silver@ssdatalakeeteproject.dfs.core.windows.net/orders"